**LoadData**

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder



df_actual = pd.read_csv('InformationByTokenClass.csv')
DATA_PATH = ''
years = ['2019', '2020', '2021', '2022', '2023','2024']
df_per_year = []
def load_data(year):
  folder_path = DATA_PATH + year + '/'
  print("Cargando data desde " + folder_path)
  csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
  dfs = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
  df_per_year.append(dfs)

for year in years:
  load_data(year)
df_per_year = [dataset for sublist in df_per_year for dataset in sublist]

Cargando data desde 2019/
Cargando data desde 2020/
Cargando data desde 2021/
Cargando data desde 2022/
Cargando data desde 2023/
Cargando data desde 2024/


**Functions**

In [2]:
#functions

# Función para crear un dataset con los tokens presentes en el diccionario iterando en la lista de datasets
def filter_and_combine_datasets(name, symbol, datasets):
    filtered_dfs = []
    
    for dataset in datasets:
        filtered_df = dataset[(dataset['name'] == name) & (dataset['symbol'] == symbol)]
        
        if not filtered_df.empty:
            filtered_dfs.append(filtered_df)

    if not filtered_dfs:
        print(f"No se encontraron datos para {name} con símbolo {symbol}.")
        return None
    
    combined_df = pd.concat(filtered_dfs, ignore_index=True)
    combined_df = combined_df.sort_values(by='last_updated').reset_index(drop=True)
    combined_df = combined_df.drop(columns=['maxSupply'], errors='ignore')
    combined_df=cleanData(combined_df)

    return combined_df


def print_high_null_columns(df, threshold=0.8):
    """
    Imprime las columnas que tienen más del 'threshold' de valores nulos.
    
    :param df: DataFrame a analizar
    :param threshold: Umbral para detectar columnas con alto porcentaje de nulos
    """
    null_percentage = df.isna().mean()
    high_null_columns = null_percentage[null_percentage > threshold]

    if not high_null_columns.empty:
        print("Columnas con más del {:.0%} de valores nulos:".format(threshold))
        print(high_null_columns)
    else:
        print("Ninguna columna tiene más del {:.0%} de valores nulos.".format(threshold))


def count_nulls(df):
  nan_counts_by_group = df.groupby('name').apply(lambda x: x.isna().sum())
  return nan_counts_by_group

def cleanData(dataset):
    df_final = dataset.replace(0, np.nan)
    print_high_null_columns(df_final, threshold=0.7)
    df_final = fill_nulls(df_final)
    return df_final

def fill_nulls(df):
    df['last_updated'] = pd.to_datetime(df['last_updated'])
    df['dateAdded'] = pd.to_datetime(df['dateAdded'])
    filled_data = []

    for name, group in df.groupby('name'):
        numeric_cols = ['circulatingSupply', 'volume24h', 'marketCap',
                        'percentChange1h', 'percentChange24h', 'percentChange7d']
        group[numeric_cols] = group[numeric_cols].apply(pd.to_numeric, errors='coerce')

        # Eliminar filas donde marketCap es NaN
        group = group.dropna(subset=['marketCap'])

        for col in numeric_cols:
            group[col] = group[col].interpolate(method='linear', limit_direction='both')

        group[numeric_cols] = group[numeric_cols].fillna(method='ffill').fillna(method='bfill')

        filled_data.append(group)
    
    filled_df = pd.concat(filled_data, ignore_index=True)
    return filled_df




**Low Market Cap Predict**

In [9]:
token_class=pd.read_csv('InformationByTokenClass.csv')
# Filtrar las monedas cuyo market cap es de 2 millones o menos
filtered_df = token_class[(token_class['Market_Cap'] <= 1_000_000) & (token_class['Market_Cap'] > 0)]
filtered_df

,Ranking,Name,Token,Price,One_Hour_Percentage,TwentyFour_Hour_Percentage,Seven_Days_Percentage,Market_Cap,URL,socialNetworks,...,Class,Capture_Date,halving_1_RankIndex,halving_1_plus250_RankIndex,halving_2_RankIndex,halving_2_plus250_RankIndex,halving_3_RankIndex,halving_3_plus250_RankIndex,halving_4_RankIndex,halving_survive
156,1798,Uno Re,UNO,0.011590,0.06,0.06,7.68,906638,https://coinmarketcap.com/currencies/unore/,"𝕏, Twitter, Telegram",...,0,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,1342.0,0
157,1811,ApeBond,ABOND,0.005992,0.50,0.63,2.20,864511,https://coinmarketcap.com/currencies/apebond/,"𝕏, Twitter, Reddit, Telegram, Chat",...,0,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,1368.0,0
158,1822,GoCrypto Token,GOC,0.003340,0.02,4.58,3.18,819189,https://coinmarketcap.com/currencies/gocrypto-...,Telegram,...,0,2024-09-23,NaN,NaN,NaN,NaN,796.0,650.0,1556.0,1
159,1824,Lithium,LITH,0.000156,1.61,0.40,17.77,818201,https://coinmarketcap.com/currencies/lithium/,"𝕏, Twitter",...,0,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,1140.0,0
160,1856,AIgentX,AIX,0.009806,0.88,6.87,66.05,728116,https://coinmarketcap.com/currencies/aigentx/,"𝕏, Twitter, Discord, Telegram",...,0,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,2732.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,5848,XTV,XTV,0.021000,0.00,10.21,7.91,209988,https://coinmarketcap.com/currencies/xtv/,"𝕏, Twitter, Telegram",...,3,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2233,5858,Cats Of Sol,COS,0.000800,0.00,1.08,9.17,79960,https://coinmarketcap.com/currencies/cats-of-sol/,"𝕏, Twitter, Telegram",...,3,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,3599.0,0
2234,5864,Baby Sora,BABYSORA,0.012124,0.01,4.01,66.79,51903,https://coinmarketcap.com/currencies/baby-sora-/,"𝕏, Twitter, Telegram",...,3,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,5324.0,0
2235,5869,TieDan,TIEDAN,0.000085,2.43,9.97,25.30,84636,https://coinmarketcap.com/currencies/tiedan/,"𝕏, Twitter, Telegram",...,3,2024-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Crear una lista vacía para almacenar los DataFrames
df_list = []

# Iterar sobre cada fila del dataframe filtrado
for index, row in filtered_df.iterrows():
    name = row['Name']
    token = row['Token']
    
    # Filtrar y combinar datos usando la función proporcionada
    combined_data = filter_and_combine_datasets(name, token, df_per_year)
    
    # Agregar el DataFrame combinado a la lista
    df_list.append(combined_data)

# Mostrar los primeros 5 elementos de la lista para verificar
df_list[:5]

In [ ]:
import pandas as pd

# Supongamos que df_list contiene los DataFrames

# Filtrar los DataFrames que no sean None y tengan 30 filas o más
filtered_df_list = [df for df in df_list if df is not None and df.shape[0] >= 15]

# Mostrar el número de DataFrames que cumplen la condición
print(f"El número de datasets con 10 filas o más es: {len(filtered_df_list)}")

# Usar un set para almacenar el nombre y símbolo de las monedas de manera única
name_symbol_set = set()

# Iterar sobre los DataFrames filtrados y extraer el nombre y símbolo
for df in filtered_df_list:
    if 'name' in df.columns and 'symbol' in df.columns:
        for index, row in df.iterrows():
            # Añadir nombre y símbolo al set (esto elimina duplicados automáticamente)
            name_symbol_set.add((row['name'], row['symbol']))

# Crear un DataFrame con los datos únicos
name_symbol_df = pd.DataFrame(list(name_symbol_set), columns=['Name', 'Symbol'])

# Mostrar el DataFrame resultante
print(name_symbol_df)

# Guardar el DataFrame en un archivo CSV
name_symbol_df.to_csv('monedas_info_dataset.csv', index=False)

print("Datos únicos guardados en 'monedas_info_dataset.csv'")


**Filtrar por categoria**

In [15]:
# Crear un diccionario con las combinaciones de 'Name' y 'Token' y su respectiva 'Class' desde filtered_df
name_token_to_class = dict(zip(zip(filtered_df['Name'], filtered_df['Token']), filtered_df['Class']))

# Agregar la columna 'Class' a name_symbol_df basada en la coincidencia de 'Name' y 'Symbol'
name_symbol_df['Class'] = name_symbol_df.apply(
    lambda row: name_token_to_class.get((row['Name'], row['Symbol']), None),
    axis=1
)

# Mostrar el DataFrame actualizado con la columna 'Class'
print(name_symbol_df)

                     Name     Symbol  Class
0                 Kambria        KAT      0
1        Wrapped Dogecoin      WDOGE      3
2                  POLKER        PKR      1
3                  Arcona     ARCONA      0
4    Castle of Blackwater       COBE      1
..                    ...        ...    ...
183               Nutcoin        NUT      3
184              The QWAN       QWAN      1
185               AIgentX        AIX      0
186          KittenWifHat  KITTENWIF      3
187            Nvidia Inu       NINU      3

[188 rows x 3 columns]


In [11]:
# Contar cuántos de cada categoría hay en la columna 'Class'
category_counts = name_symbol_df['Class'].value_counts()

# Mostrar el resultado
print(category_counts)



Class
1    92
3    60
0    31
2     5
Name: count, dtype: int64


In [ ]:
import pandas as pd

# Iterar por las filas de name_symbol_df
def process_by_name_symbol(df_per_year, name_symbol_df):
    # Estructuras para almacenar resultados y errores
    results = []
    errors = []
    
    # Iterar por cada fila de `name_symbol_df`
    for _, row in name_symbol_df.iterrows():
        name = row["Name"]
        token = row["Symbol"]
        class_category = row["Class"]

        print(f"Procesando: Name={name}, Token={token}, Class={class_category}")

        try:
            # Filtrar y combinar datasets
            df_filtered = filter_and_combine_datasets(name, token, df_per_year)

            if df_filtered is None or df_filtered.empty:
                print(f"No se encontraron datos para {name} con símbolo {token}.")
                errors.append({"name": name, "token": token, "error": "No se encontraron datos"})
                continue

            # Realizar predicción con SARIMA
            graph_data, graph_layout, comparison_results, one_month_predict = train_sarima_and_predict(df_filtered)

            # Guardar los resultados en una lista
            results.append({
                "name": name,
                "token": token,
                "class": class_category,
                "graph_data": graph_data,
                "graph_layout": graph_layout,
                "comparison_results": comparison_results,
                "one_month_predict": one_month_predict
            })

            print(f"Predicción completada para {name} ({token}) con éxito.")
        
        except Exception as e:
            print(f"Error procesando {name} ({token}): {str(e)}")
            errors.append({"name": name, "token": token, "error": str(e)})

    # Retornar resultados y errores
    return results, errors

# Ejecución principal
results, errors = process_by_name_symbol(df_per_year, name_symbol_df)

# Manejo de resultados
if results:
    print(f"Se procesaron correctamente {len(results)} monedas.")
    # Convertir los resultados en un DataFrame
    results_df = pd.DataFrame(results)

    # Seleccionar el top 5 de cada categoría basado en "one_month_predict"
    top_5_per_class = (
        results_df.sort_values(by="one_month_predict", ascending=False)
        .groupby("class")
        .head(5)
    )

    # Guardar el top 5 por categoría en un archivo CSV
    top_5_per_class.to_csv("top_5_per_class.csv", index=False)
    print("Archivo 'top_5_per_class.csv' guardado con éxito.")

# Manejo de errores
if errors:
    print(f"Se encontraron errores en {len(errors)} monedas.")
    errors_df = pd.DataFrame(errors)
    errors_df.to_csv("errors.csv", index=False)
    print("Archivo 'errors.csv' guardado con éxito.")


In [17]:
# Manejo de resultados
if results:
    print(f"Se procesaron correctamente {len(results)} monedas.")
    # Convertir los resultados en un DataFrame
    results_df = pd.DataFrame(results)

    # Seleccionar el top 5 de cada categoría basado en "one_month_predict"
    top_5_per_class = (
        results_df.sort_values(by="one_month_predict", ascending=False)
        .groupby("class")
        .head(5)
    )

    # Guardar el top 5 por categoría en un archivo CSV
    top_5_per_class.to_csv("top_5_per_class.csv", index=False)
    print("Archivo 'top_5_per_class.csv' guardado con éxito.")

# Manejo de errores
if errors:
    print(f"Se encontraron errores en {len(errors)} monedas.")
    errors_df = pd.DataFrame(errors)
    errors_df.to_csv("errors.csv", index=False)
    print("Archivo 'errors.csv' guardado con éxito.")

Se procesaron correctamente 153 monedas.
Archivo 'top_5_per_class.csv' guardado con éxito.
Se encontraron errores en 35 monedas.
Archivo 'errors.csv' guardado con éxito.


**Model, Load LigthBGM** 

In [102]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import load
import plotly.graph_objects as go
from datetime import datetime
from skforecast.plot import set_dark_theme
from skforecast.preprocessing import exog_long_to_dict

df = pd.read_csv('df_skforecast.csv')
forecaster = load('skforecaster_model.joblib')

df['last_updated'] = pd.to_datetime(df['last_updated'])
names = unique_names = df['name'].unique().tolist()

def predictLight(name, steps_p, df):
    # Filtrar los datos del token específico
    df_pred = df[df['name'] == name]
    df_pred = df_pred.sort_values(by='last_updated')

    # Preparar la serie y las variables exógenas
    series = df_pred[['name', 'last_updated', 'marketCap']]
    exog = df_pred[['name', 'last_updated', 'cmcRank', 'price', 'volume24h', 'percentChange1h', 'percentChange24h']]

    # Convertir las variables exógenas a diccionario para el modelo
    exog_dict = exog_long_to_dict(
        data=exog,
        series_id='name',
        index='last_updated',
        freq='W'  # Frecuencia semanal
    )

    # Realizar la predicción para los próximos 'steps_p' periodos
    predicciones = forecaster.predict(steps=steps_p, exog=exog_dict, suppress_warnings=True)
    predicciones.index = pd.date_range(start=df_pred['last_updated'].max(), periods=steps_p, freq='W')

    # Filtrar las predicciones solo para el token seleccionado
    if name in predicciones.columns:
        predicciones = predicciones[[name]]
    else:
        raise ValueError(f"No se encontraron predicciones para el token: {name}")

    # Crear la gráfica interactiva con Plotly
    fig = Figure()

    # Graficar datos reales
    fig.add_trace(Scatter(
        x=series['last_updated'],
        y=series['marketCap'],
        mode='lines',
        name='Real',
        line=dict(color='blue')
    ))


    # Cálculo de los cambios porcentuales para las predicciones futuras
    last_real_value = series['marketCap'].iloc[-1]  # Último valor real
    percent_changes = ((predicciones[name].values - last_real_value) / last_real_value) * 100

    # Preparamos los datos de comparación futura
    future_dates = pd.date_range(start=predicciones.index[0], periods=steps_p, freq='W')
    future_comparison = pd.DataFrame({
        'ds': future_dates,
        'y_actual': ['-'] * len(future_dates),
        'y_pred': predicciones[name].values,
        'abs_error': ['-'] * len(future_dates),
        'percent_error': ['-'] * len(future_dates),
        'percent_change': percent_changes.tolist(),
        'name': name
    })

    # Agregar la línea de la predicción final (predicción a futuro con el modelo completo)
    fig.add_trace(Scatter(
        x=future_dates,
        y=predicciones[name].values,
        mode='lines',
        name='Prediction',
        line=dict(color='#d1a800', width=2)
    ))

    # Personalizar el diseño del gráfico
    fig.update_layout(
        title=f'Comparación Predicción vs Real para {name}',
        xaxis_title='Fecha',
        yaxis_title='Market Cap',
        legend_title='Tipo',
        template='plotly_white',
        xaxis=dict(tickformat='%Y-%m-%d')  # Formato para que solo aparezca la fecha (sin hora)
    )

    # Convertir los datos a formatos serializables para JSON
    graph_data = [trace.to_plotly_json() for trace in fig.data]
    graph_layout = fig.layout.to_plotly_json()

    # Convertir todos los objetos de numpy a listas
    comparison_results = future_comparison.to_dict(orient='records')

    # Serializar los datos
    graph_data_serializable = convert_to_serializable(graph_data)
    graph_layout_serializable = convert_to_serializable(graph_layout)
    comparison_results_serializable = convert_to_serializable(comparison_results)

    # Calcular el cambio porcentual para la predicción a 1 mes
    one_month_predict = percent_changes[3] if len(percent_changes) > 3 else 0.0

    return graph_data_serializable, graph_layout_serializable, comparison_results_serializable, one_month_predict

SARIMAX


In [ ]:
from flask import Flask, request, jsonify
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from pmdarima import auto_arima
from plotly.graph_objects import Scatter, Figure
from plotly.offline import plot
import pandas as pd
import numpy as np
from datetime import datetime
from flask_cors import CORS

# Crear la app de Flask
app = Flask(__name__)
CORS(app)
# Función para dividir los datos en entrenamiento y prueba
def split_data_by_date(df, train_size=0.8):
    df_sorted = df.sort_values(by='ds')
    cutoff_index = int(len(df_sorted) * train_size)
    train_data = df_sorted[:cutoff_index]
    test_data = df_sorted[cutoff_index:]
    return train_data, test_data

# Función para determinar si la serie temporal tiene suficiente longitud para capturar estacionalidad
def get_seasonality(df):
    if len(df) < 12:  # Menos de 12 puntos de datos, no se considera estacionalidad
        return False
    return True  # Para más de 12 puntos de datos, se considera estacionalidad

# Función para convertir NaN a None y ndarray a list
def convert_to_serializable(obj):
    """Convierte NaN a None y ndarray a list para ser serializado a JSON"""
    if isinstance(obj, float) and (obj != obj):  # Verifica NaN (NaN != NaN)
        return None
    elif isinstance(obj, np.ndarray):  # Convierte ndarray a lista
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    else:
        return obj
def train_sarima_and_predict(df, future_months=3, prediction_interval='7D'):
    # Agrupar por fecha y calcular el promedio de marketCap
    df_grouped = df.groupby(['name', 'last_updated']).agg({
        'marketCap': 'mean',
        'price': 'mean',
        'volume24h': 'mean',
        'cmcRank': 'mean',
        'percentChange1h': 'mean',
        'percentChange24h': 'mean'
    }).reset_index()

    df_grouped.rename(columns={'last_updated': 'ds', 'marketCap': 'y'}, inplace=True)

    # Verificar si hay valores nulos antes de aplicar el escalado
    exogenous_features = ['price', 'volume24h', 'cmcRank', 'percentChange1h', 'percentChange24h']
    missing_columns = df_grouped[exogenous_features].isnull().sum()
    if missing_columns.any():
        return jsonify({"error": f"Faltan valores en las columnas: {', '.join(missing_columns[missing_columns > 0].index)}"}), 400

    # Verificar si la serie temporal (y) tiene nulos
    if df_grouped['y'].isnull().sum() > 0:
        return jsonify({"error": "Faltan valores en la columna 'y' (marketCap)."}), 400

    # Aplicar transformación logarítmica para estabilizar la varianza
    df_grouped['y'] = np.log1p(df_grouped['y'])

    # Escalar las variables exógenas
    scaler = StandardScaler()
    df_grouped[exogenous_features] = scaler.fit_transform(df_grouped[exogenous_features])

    # Dividir los datos en entrenamiento y prueba (80% - 20%)
    train_size = int(len(df_grouped) * 0.8)
    train_data = df_grouped[:train_size]
    test_data = df_grouped[train_size:]

    # Verificar si tenemos datos suficientes para realizar el modelado
    if len(test_data) == 0:
        return jsonify({"error": "No hay suficientes datos en el conjunto de prueba para realizar las predicciones."}), 400

    # Determinar si se debe usar estacionalidad dependiendo del tamaño de los datos
    seasonal_flag = get_seasonality(train_data)

    # Optimizar los parámetros con auto_arima
    model_auto = auto_arima(
        train_data['y'], 
        exogenous=train_data[exogenous_features], 
        seasonal=seasonal_flag,
        m=12 if seasonal_flag else 1,
        stepwise=True, 
        trace=True
    )
    print(model_auto.summary())

    # Ajustar el modelo SARIMA con los mejores parámetros encontrados
    model = SARIMAX(
        train_data['y'],
        exog=train_data[exogenous_features],
        order=model_auto.order,
        seasonal_order=model_auto.seasonal_order if seasonal_flag else (0, 0, 0, 0),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    model_fit = model.fit(disp=False)

    # Hacer predicciones en el conjunto de prueba
    test_forecast = model_fit.get_forecast(steps=len(test_data), exog=test_data[exogenous_features])
    y_pred = np.expm1(test_forecast.predicted_mean)  # Revertir transformación logarítmica

    # Calcular MAE y RMSE
    y_test_actual = np.expm1(test_data['y'])
    mae = mean_absolute_error(y_test_actual, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))
    print(f"Mean Absolute Error (Prueba): {mae}")
    print(f"Root Mean Squared Error (Prueba): {rmse}")

    # Crear DataFrame de comparación
    comparison = pd.DataFrame({
        'ds': test_data['ds'],
        'y_actual': y_test_actual.tolist(),
        'y_pred': y_pred.tolist(),
        'abs_error': abs(y_test_actual - y_pred).tolist(),
        'percent_error': ((abs(y_test_actual - y_pred) / y_test_actual) * 100).tolist(),
        'name': df['name'].iloc[0]
    })
    
    # Asegúrate de que el número de pasos futuros sea el correcto
    future_steps = (future_months * 4)  # Aproximadamente 3 meses (12 semanas)

    # Verifica si tienes suficientes pasos para la predicción
    if len(test_data) >= future_steps:
        # Si tienes suficiente longitud en los datos de prueba, usa los exógenos directamente
        exog_futuro = test_data[exogenous_features].iloc[-future_steps:]
    else:
        # Si no tienes suficiente longitud, usa el último valor disponible y repítelo para completar los pasos necesarios
        exog_futuro = pd.DataFrame(np.tile(test_data[exogenous_features].iloc[-1].values, (future_steps, 1)),
                                columns=exogenous_features)

    # Asegúrate de que las dimensiones de los exógenos futuros sean correctas
    if exog_futuro.shape[0] != future_steps:
        return jsonify({"error": f"El número de pasos futuros no coincide con los exógenos."}), 400

    # Generar la predicción para los pasos futuros con los exógenos correctos
    future_forecast = model_fit.get_forecast(steps=future_steps, exog=exog_futuro)

    # Generar fechas futuras
    future_dates = pd.date_range(start=test_data['ds'].max(), periods=future_steps + 1, freq=prediction_interval)[1:]

    # Predicciones futuras
    future_predictions = np.expm1(future_forecast.predicted_mean)

    # Calcular el percentChange basado en el último valor real
    last_real_value = y_test_actual.iloc[-1]  # Último valor real



    # Crear la gráfica interactiva con Plotly
    fig = Figure()

    # Graficar datos reales (todas las fechas reales de entrenamiento + prueba)
    all_real_data = pd.concat([train_data[['ds', 'y']], test_data[['ds', 'y']]], ignore_index=True)
    fig.add_trace(Scatter(x=all_real_data['ds'], y=np.expm1(all_real_data['y']), mode='lines', name='Real', line=dict(color='blue')))

    # Graficar las predicciones solo desde el inicio del conjunto de prueba
    fig.add_trace(Scatter(x=comparison['ds'], y=comparison['y_pred'], mode='lines', name='Predicción', line=dict(color='red', dash='dash')))

    # Graficar el modelo entrenado con todo el conjunto de datos
    model_all_data = SARIMAX(
        df_grouped['y'],
        exog=df_grouped[exogenous_features],
        order=model_auto.order,
        seasonal_order=model_auto.seasonal_order if seasonal_flag else (0, 0, 0, 0),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    model_all_fit = model_all_data.fit(disp=False)
    all_forecast = model_all_fit.get_forecast(steps=future_steps, exog=exog_futuro)
    future_predictions_all = np.expm1(all_forecast.predicted_mean)


    percent_changes = ((future_predictions_all - last_real_value) / last_real_value) * 100

    one_month_predict = 0.0
    # Imprimir el percentChange para las predicciones futuras
    for i, percent_change in enumerate(percent_changes):
        print(f"Percent Change para la predicción futura {i+1} ({future_dates[i]}): {percent_change:.2f}%")
        if i == 3:
            one_month_predict = percent_change


        # Crear DataFrame con las predicciones futuras
    future_comparison = pd.DataFrame({
        'ds': future_dates,
        'y_actual': ['-'] * len(future_dates),
        'y_pred': [
            future_predictions_all.tolist()[i] if '-' == '-' else future_predictions[i]
            for i in range(len(future_dates))
        ],
        'abs_error': ['-'] * len(future_dates),
        'percent_error': ['-'] * len(future_dates),  # No calcular el error porcentual en las predicciones futuras
        'percent_change': percent_changes.tolist(),  # Incluir percent_change en las predicciones futuras
        'name': df['name'].iloc[0]
    })
    # Unir los resultados de la comparación
    comparison = pd.concat([comparison, future_comparison], ignore_index=True)

    # Agregar la línea amarilla (modelo entrenado con todo el conjunto de datos) con un tono más oscuro
    fig.add_trace(Scatter(x=future_dates, y=future_predictions_all, mode='lines', name='Prediction(Train+Test)', line=dict(color='#d1a800', width=2)))

    fig.update_layout(
        title=f'Comparación Predicción vs Real para {df["name"].iloc[0]}',
        xaxis_title='Fecha',
        yaxis_title='Market Cap',
        legend_title='Tipo',
        template='plotly_white',
        xaxis=dict(
            tickformat='%Y-%m-%d'  # Formato para que solo aparezca la fecha (sin hora)
        )
    )

    # Serializar datos y layout para JSON
    graph_data = [trace.to_plotly_json() for trace in fig.data]
    graph_layout = fig.layout.to_plotly_json()

    # Convertir todos los objetos de numpy a listas
    comparison_results = comparison.to_dict(orient='records')

    # Convertir todo a estructuras serializables para JSON
    graph_data_serializable = convert_to_serializable(graph_data)
    graph_layout_serializable = convert_to_serializable(graph_layout)
    comparison_results_serializable = convert_to_serializable(comparison_results)
    print(one_month_predict)
    # Ahora `comparison_results_serializable` contiene el `percent_change`
    return graph_data_serializable, graph_layout_serializable, comparison_results_serializable, one_month_predict

# Rutas de Flask
@app.route('/', methods=['GET'])
def index():
    return "Bienvenido al servidor Flask!"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()

        if 'name' not in data or 'symbol' not in data:
            return jsonify({"error": "Faltan parámetros 'name' o 'symbol'"}), 400

        name = data['name']
        symbol = data['symbol']
        print(f"Recibidos: name={name}, symbol={symbol}")

        # Aquí deberías tener el DataFrame `df_per_year`
        df_final = filter_and_combine_datasets(name, symbol, df_per_year)

        if df_final is None or df_final.empty:
            return jsonify({"error": f"No se encontraron datos para {name} con símbolo {symbol}."}), 404

        # Preparar los datos
        df_final['last_updated'] = pd.to_datetime(df_final['last_updated'], errors='coerce')

        # Generar predicciones y obtener datos para la gráfica
        graph_data, graph_layout, comparison_results,one_month_predict = train_sarima_and_predict(df_final)

        return jsonify({
            "graph_data": graph_data,  # Datos de la gráfica serializados
            "graph_layout": graph_layout,  # Layout de la gráfica serializado
            "comparison_results": comparison_results,# Resultados de comparación
            "one_month_predict": one_month_predict  # Predicción para un mes  
        }), 200

    except Exception as e:
        print(f"Error inesperado: {str(e)}")
        return jsonify({"error": f"Error inesperado: {str(e)}"}), 500


@app.route('/top_5_per_class', methods=['GET'])
def get_top_5_per_class():
    try:
        # Ruta del archivo CSV
        file_path = 'top_5_per_class.csv'

        # Verificar si el archivo existe
        if not os.path.exists(file_path):
            return jsonify({"error": f"El archivo {file_path} no se encuentra."}), 404

        # Leer el archivo CSV
        df = pd.read_csv(file_path)

        # Convertir a JSON
        data_json = df.to_dict(orient='records')

        return jsonify(data_json), 200

    except Exception as e:
        print(f"Error al procesar el archivo: {str(e)}")
        return jsonify({"error": f"Error inesperado: {str(e)}"}), 500


@app.route('/predict-l', methods=['POST'])
def approute():
    try:
        data = request.get_json()
        namecoin = data['name']
        df = pd.read_csv('df_skforecast.csv')
        df['last_updated'] = pd.to_datetime(df['last_updated'], errors='coerce')

        
        graph_data, graph_layout, comparison_results, one_month_predict = predictLight(namecoin, 30, df)


        response = {
            'graph_data': graph_data,
            'graph_layout': graph_layout,
            'comparison_results': comparison_results,
            'one_month_predict': one_month_predict
        }

        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)}), 400


# Ejecutar Flask
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5000)
